In [1]:
%load_ext autoreload
%autoreload 2

import warnings
import os 
warnings.filterwarnings('ignore') 

from tqdm import tqdm
import numpy as np
from matplotlib import pyplot as plt 
from sklearn.gaussian_process.kernels import WhiteKernel, ConstantKernel, Matern, RBF
from sklearn.preprocessing import MinMaxScaler


os.chdir("../")
plt.style.use("mpl_style/matplotlib.rc")

from src.utils import XY_from_csv, random_sampling_no_replace
from src.acquisition import run_acquisition
from src.algorithms import MultibandUnion, MultibandUnionIntersection, GlobalOptimization1D, ParetoFront, Wishlist, PercentileSet1D, MonodisperseLibrary
from src.models import MGPR, fit_hypers
from src.metrics import get_n_obtained, get_jaccard_posterior
from src.plotting import plot_final_metrics, plot_iteration_results


In [2]:
estimated_y_range = [[0, 30], [0,30]]

X, Y = XY_from_csv("datasets/np_synthesis.csv", columns_x=["c1", "c2", "pH", "T"], columns_y=["size", "dispersity"])

x_scaler = MinMaxScaler(feature_range=(0, 1))
y_scaler = MinMaxScaler(feature_range=(-1, 1))

y_scaler.fit(estimated_y_range)
scalers = [x_scaler, y_scaler]

X = x_scaler.fit_transform(X)
Y = y_scaler.fit_transform(Y)

n_features = X.shape[1]
n_properties = Y.shape[1]

# handles one-property measurements
if len(Y.shape) == 1:
    Y = Y.reshape(-1, 1)

In [3]:
# algorithm = MultiRegionSetIntersection(threshold_list = [[0.0, 20.0], [0.0, 2.5]], scalers = scalers)

user_algo_params = {'scalers': scalers,
                    'target_radii_list': [6, 8,  12, 14, 16, 18, 20, 25],
                    'polysdispersity_threshold': 5.0,
                    'target_radii_tol':0.5}

algorithm = MonodisperseLibrary(user_algo_params=user_algo_params)



# algorithm = Wishlist(
#     threshold_bounds=[[[0.0, 2.0], [0.0, 0.2]], [[3.0, 4.0], [0.3, 0.4]], [[8.0, 10.0], [0.0, 0.1]]], scalers=scalers
# )

# algorithm = PercentileSet(percentile_threshold=95, scalers=scalers)
# algorithm = ParetoFront(tolerance_list = [1.0, 1.0],  max_or_min_list = [0, 0], scalers = scalers)

In [5]:
all_ids = list(np.arange(0, len(X)))  # integer mapping design space
true_target_ids = algorithm.identify_subspace(
    f_x=Y, x=X
)  # ground truth set of design points which achieve experimental goal

In [6]:
plotting = True
prevent_requery = True
plot_frequency = 50
n_posterior_samples = 15 # relevant for InfoBAX and mixedBAX 
n_initial = 1 # Number of initial datapoints 
n_iters = 201 # Number of measurements to be performed 
n_repeats = 1 # Repeats with different dataset initializations 
fixed_hypers = False 
adaptive_fit_freq = 10

kernel_initial = ConstantKernel(constant_value=1.0, constant_value_bounds=[0.01, 3.0]) * Matern(nu = 5/2, length_scale= n_features * [1.0], length_scale_bounds= n_features * [[0.01, 3.0]]) + WhiteKernel(noise_level=0.01, noise_level_bounds='fixed')

kernel_initial_list = n_properties * [kernel_initial]

In [7]:

metrics = {
    "SwitchBAX": {"n_obtained": [], "jaccard_posterior_index": [], "switch_strategy": []},
    "US": {"n_obtained": [], "jaccard_posterior_index": [], "switch_strategy": []},
    "MeanBAX": {"n_obtained": [], "jaccard_posterior_index": [], "switch_strategy": []},
    "InfoBAX": {"n_obtained": [], "jaccard_posterior_index": [], "switch_strategy": []},
}

# Baseling for random sampling without replacement (expectation of hypergeometric distribution)
random_sampling = [random_sampling_no_replace(len(X), len(true_target_ids), n) for n in range(n_initial, n_iters)]

# Baseline for best possible acquisition (i.e. acquire a target point at each iteration; need an "oracle" to do this)
if n_iters <= len(true_target_ids):
    best_possible_n_obtained = np.arange(n_initial, n_iters + n_initial)
else:
    best_possible_n_obtained = list(np.arange(n_initial, len(true_target_ids))) + list(
        len(true_target_ids) * np.ones(n_iters + n_initial - len(true_target_ids))
    )

# Acquisition functions that use BAX for subset estimation
# strategies = ["MeanBAX", "SwitchBAX", "US", "InfoBAX"]
strategies = ["MeanBAX", "SwitchBAX", "US"]

# Calculate hypers based on the entire dataset; this is not possible in a real experiment but allows us to compare acquisition fn to acquisition fn
if fixed_hypers:
    kernel_list = fit_hypers(x_train=X, y_train=Y, kernel_list=kernel_initial_list, n_restarts_optimizer=1)

for strategy in strategies:
    for j in range(n_repeats):  # to see variance w.r.t initial datapoint choice
        np.random.seed(j) # make sure all strategies get same initial points
        train_indices = list(np.random.choice(all_ids, n_initial))
        x_train = X[train_indices]
        y_train = Y[train_indices]

        collected_ids = list(train_indices)
        n_obtained_list = []
        jaccard_posterior_list = []
        switch_list = [] 

        for i in tqdm(range(n_iters)):
            # Adaptive hyperparameter fitting
            if (i % adaptive_fit_freq == 0) and (fixed_hypers == False):
                kernel_list = fit_hypers(x_train=x_train, y_train=y_train, kernel_list=kernel_initial_list)
            
            # Define GP model with fixed, fitted hypers. Note, we need this so that all the n_posterior models for InfoBAX have the same kernel
            multi_gpr = MGPR(kernel_list=kernel_list)

            # Acquire next index
            x_train, y_train, model, collected_ids, acquisition_function, switch_strategy = run_acquisition(
                x_train, y_train, X, Y, strategy, algorithm, multi_gpr, collected_ids, n_posterior_samples
            )

            # Calculate metrics
            posterior_mean, posterior_std = model.predict(X)
            predicted_target_ids = algorithm.identify_subspace(f_x=posterior_mean, x=X)
            n_obtained_list.append(get_n_obtained(collected_ids, true_target_ids))
            jaccard_posterior_list.append(get_jaccard_posterior(predicted_target_ids, true_target_ids))
            switch_list.append(switch_strategy)

            if (i % plot_frequency == 0) and (plotting) and (i != 0):
                plot_iteration_results(
                    X,
                    Y,
                    x_scaler,
                    y_scaler,
                    collected_ids,
                    true_target_ids,
                    predicted_target_ids,
                    acquisition_function,
                    n_obtained_list,
                    jaccard_posterior_list,
                    best_possible_n_obtained,
                    random_sampling,
                    n_initial
                )

        metrics[strategy]["n_obtained"].append(n_obtained_list)
        metrics[strategy]["jaccard_posterior_index"].append(jaccard_posterior_list)
        metrics[strategy]["switch_strategy"].append(switch_list)

        plt.figure(figsize=(10,3))
        plt.step(np.arange(n_initial, n_initial + n_iters), metrics[strategy]['switch_strategy'][0])
        plt.ylabel('Exploit/Explore')
        plt.xlabel('Dataset Size')
        plt.show()

plot_final_metrics(n_iters, metrics, strategies, best_possible_n_obtained, random_sampling)

  8%|▊         | 17/201 [00:03<00:31,  5.88it/s]